# Haar Compression Activity
stough 202-

DIP 6, 8.9

In this activity you're going to use the Haar wavelet block transform and lossy compression in two different contexts

- A single $4x4$ single-channel "image" block with fairly high contrast.
- A larger color image with $8x8$ blocks.

In each case we're going to 

- Transform: Turn a block of image pixels into a block of Haar wavelet coefficients. 
- Compress: We're going to effectively compress by zeroing out insigficant/small coefficients.
- Reconstruct: Take the "compressed" coefficent block and reconstruct the block of image pixels, with some loss.

Recall from [`view_basis_blocks`](./view_basis_blocks.ipynb) that the Haar $4x4$ transform matrix is like so:

\begin{equation*}
\mathbf{H_{Haar}} =  \frac{1}{2}\begin{vmatrix}
1 & 1 & 1 & 1\\
1 & 1 & -1 & -1 \\
{\sqrt 2} & -{\sqrt 2} & 0 & 0 \\
0 & 0 & {\sqrt 2} & -{\sqrt 2}
\end{vmatrix}
\end{equation*}

To transform a block of pixels $\mathbf{B}$ into a block of coefficients:

\begin{equation*}
\mathbf{T} = \mathbf{H}\times\mathbf{B}\times\mathbf{H}^T
\end{equation*}

where $\mathbf{H}^T$ is the transpose. Through the above we basically replace the set of independent pixels with the set of coefficients with respect to the Haar basis patterns, as we'll see with `vis_blocks`. Once we have $\mathbf{T}$, we can quantize it to the degree that we wish, zeroing out small coefficients, potentially replacing large coefficients with nearby approximations on a standard grade (e.g. replace anything in $[236,276]$ with $256$). 

To reconstruct the original $\mathbf{B}$, we can simply

\begin{equation*}
\mathbf{B} = \mathbf{H}^T\times\mathbf{T}\times\mathbf{H}
\end{equation*}

## Imports
We're going to use [`wavelet_utils`](../dip_utils/wavelet_utils.py).

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair,
                       vis_surface)

from wavelet_utils import (make_haar_matrix,
                           make_random_basis,
                           make_klt_basis,
                           make_dct_matrix,
                           make_standard_matrix,
                           vis_blocks)

## 1. $4\times4$ Haar Wavelet Compression
Use `make_haar_matrix` and `vis_blocks` to see the 16 Haar patterns for the $\mathbf{H_{Haar}}$ matrix

In [16]:
H = make_haar_matrix(4)
vis_blocks(H)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Transform a $4\times4$ block of pixels.
We'll define the block of pixels in question. Then you transform it into the coefficient block $\mathbf{T}$. See [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) to compute $\mathbf{H}\times\mathbf{B}\times\mathbf{H}^T$.

In [4]:
B = np.array([[0,1,1,1],
              [0,1,0,0],
              [0,1,1,1],
              [0,1,0,0]])

plt.figure(figsize=(4,4))
plt.imshow(B, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
HT = H.transpose()

In [8]:
# Haar basis for 2*2 {[1,1],[1,-1]}
# H B H^T = {[1,1],[1,-1]}*{[a,b],[c,d]}*{[1,1],[1,-1]}^T = {[a+b+c+d,a-b+c-d],[a+b-c-d, a-b-c+d]}

In [17]:
T = np.matmul(H,np.matmul(B,HT)) # 16 coefficient of energy required w.r.t the basis (don't call vis_blocks, which does outer product to it)

In [18]:
T

array([[ 2.00000000e+00,  0.00000000e+00, -1.41421356e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 7.07106781e-01, -7.07106781e-01, -4.26642159e-17,
         0.00000000e+00],
       [ 7.07106781e-01, -7.07106781e-01, -4.26642159e-17,
         0.00000000e+00]])

In [21]:
plt.figure()
plt.imshow(T, cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### A. Viewing the transform coefficients in the matrix $\mathbf{T}$, what are the row and column coords for the two largest coefficients (in absolute value)?

In [23]:
H # The haar matrix

array([[ 0.5       ,  0.5       ,  0.5       ,  0.5       ],
       [ 0.5       ,  0.5       , -0.5       , -0.5       ],
       [ 0.70710678, -0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.70710678, -0.70710678]])

In [26]:
B0 = np.outer(H[0,...], H[0,...])
B1 = np.outer(H[0],H[2])

In [27]:
B1

array([[ 0.35355339, -0.35355339,  0.        ,  0.        ],
       [ 0.35355339, -0.35355339,  0.        ,  0.        ],
       [ 0.35355339, -0.35355339,  0.        ,  0.        ],
       [ 0.35355339, -0.35355339,  0.        ,  0.        ]])

In [32]:
# B = 2*B0 + -1.41*B1 + ... coefficients are observed in T
plt.figure()
plt.imshow(2*B0 - 1.41421356*B1, cmap='gray', vmin=0, vmax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
B2 = np.outer(H[2],H[0])
B2

array([[ 0.35355339,  0.35355339,  0.35355339,  0.35355339],
       [-0.35355339, -0.35355339, -0.35355339, -0.35355339],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [34]:
plt.figure()
plt.imshow(2*B0 - 1.41421356*B1 + .707106781*B2, cmap='gray', vmin=0, vmax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### B. Show the two basis blocks corresponding to those two coefficients.

### C. Show and explain as explicitly as you can how the two coefficients are what they are.

### D. Show the reconstruction of the image $\mathbf{B}$ using only those basis blocks/coefficients. 
Is this reconstruction trending in the right direction? Explain

## 2. Larger Image Block Transform and Reconstruction.
Use the code in [`block_viewing_demo`](./block_viewing_demo.ipynb) and [`basis_decomp_64pix.ipynb`](./basis_decomp_64pix.ipynb) to perform a similar transform/compress/reconstruct sequence on a larger image of your choice. 

- Use $8\times8$ blocks.
- Use [`resize`](https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=resize#skimage.transform.resize) on the image you choose to ensure that $8$ evenly divides it in height and width. This is for convenience: compression schemes have other mechanisms to deal with odd-shaped images, which we will ignore here.
- Perform reconstructions using the largest $[1,2,4,8,16]$ coefficients in absolute value per block. Note that keeping $16$ coefficients is equivalent to $\sim 3$x compression. 
- For at least the reconstructions using $[2,4]$, use `vis_pair` and zoom into an area to observe the effectiveness of the reconstruction on such limit basis (32x, 16x compression).  

You can **export this notebook** as a PDF to submit this work.